In [113]:
#Read an Excel  file and extract 3 values,

import pandas as pd

# Path to the Excel file
file_path = r"C:\Users\Randall\Downloads\Listado de Movimiento de Bancos.xls"

# Read the Excel file
data = pd.read_excel(file_path)

# Extract the values from the specified columns
columns_to_extract = ['Recibo', 'Concepto', 'Débito']
extracted_data = data[columns_to_extract].copy()

# Split 'Concepto' at the first space into 'code' and 'name'
def split_concepto(value):
    if isinstance(value, str):
        parts = value.split(' ', 1)
        if len(parts) == 2:
            return pd.Series({'code': parts[0], 'name': parts[1]})
        else:
            return pd.Series({'code': parts[0], 'name': ''})
    else:
        return pd.Series({'code': '', 'name': ''})

extracted_data[['code', 'name']] = extracted_data['Concepto'].apply(split_concepto)

# Create 'flag' column: 1 if 'Recibo' and 'code' do not match, else 0
extracted_data['flag'] = (extracted_data['Recibo'].astype(str) != extracted_data['code'].astype(str)).astype(int)

# Create 'final' column: if 'Recibo' and 'code' are equal, use 'Recibo'; else use the longest one
extracted_data['final'] = extracted_data.apply(
    lambda row: row['Recibo'] if str(row['Recibo']) == str(row['code']) else max([str(row['Recibo']), str(row['code'])], key=len),
    axis=1
)

extracted_data['final']

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


0     HMHCTBQDJ3
1     HM9F8N92TN
2     HMDXW9PN54
3     HMACKQQ9RA
4     HMFSKRMFRD
         ...    
66    HMFZMPW5RF
67    HMFQ224SCX
68    HMYXNBE4RW
69    HMYCTD22WH
70    HMM4BQKTAH
Name: final, Length: 71, dtype: object

In [114]:
import pandas as pd

# Path to the Excel file
file_path = r"C:\Users\Randall\Downloads\reservations.csv"

# Read the Excel file
data = pd.read_csv(file_path)
data['Ingresos'] = data['Ingresos'][1::]

Reservaciones = data[data['Estado'] == 'Huésped anterior']

# Display the first few rows of the dataframe

In [115]:
Reservaciones['Ingresos'].head()

1    $142.44
3    $110.85
4    $359.40
5    $251.29
7     $92.65
Name: Ingresos, dtype: object

In [116]:
# Compare 'Recibo' in extracted_data with 'Código de confirmación' in Reservaciones
# Check for matches based on 'final' and 'Código de confirmación'
matches = extracted_data['final'].isin(Reservaciones['Código de confirmación'])

# Further check for matches by comparing 'Ingresos' and 'Débito'
matches_with_price_check = extracted_data[matches].merge(
    Reservaciones[['Código de confirmación', 'Ingresos']],
    left_on='final',
    right_on='Código de confirmación'
)
matches_with_price_check = matches_with_price_check[
    matches_with_price_check['Débito'].astype(float) == matches_with_price_check['Ingresos'].str.replace('$', '').str.replace(',', '').astype(float)
]

# Count the number of matches
num_matches = len(matches_with_price_check)
print(f"Number of matches with price check: {num_matches}")

# Check for non-matches
non_matches = extracted_data[~matches]

# Further check for non-matches by comparing 'Ingresos' and 'Débito'
non_matches_with_price_check = non_matches.merge(
    Reservaciones[['Código de confirmación', 'Ingresos']],
    left_on='final',
    right_on='Código de confirmación',
    how='inner'
)
non_matches_with_price_check = non_matches_with_price_check[
    non_matches_with_price_check['Débito'].astype(float) == non_matches_with_price_check['Ingresos'].str.replace('$', '').str.replace(',', '').astype(float)
]

# Print a message if there are non-matches with price check
if not non_matches_with_price_check.empty:
    print("There are non-matches that match by price:")
    non_matches_with_price_check

Number of matches with price check: 23


In [117]:
# Filter the Reservaciones dataframe to get non-matching reservation information
non_matching_reservations_info = Reservaciones[Reservaciones['Código de confirmación'].isin(non_matching_reservaciones)]
non_matching_reservations_info


,Código de confirmación,Estado,Nombre del huésped,Contacta,Número de adultos,Número de niños,Número de bebés,Fecha de inicio,Hasta,Número de noches,Reservado,Anuncio,Ingresos
1,HMD3JTFPK9,Huésped anterior,Liseth Astudillo,+593 98 146 1964,4,1,0,14/9/2025,15/9/2025,1,2025-09-14,Aurora | Vista Increíble | Depto Lujo | Factura,$142.44
3,HMMZDW4YDD,Huésped anterior,Lorena Cuellar,NaN,3,0,0,12/9/2025,13/9/2025,1,2025-09-12,Aurora | Vista Increíble | Depto Lujo | Factura,$110.85
4,HMPCAFCWC2,Huésped anterior,Victoria Mora Martinez,NaN,3,1,0,6/9/2025,9/9/2025,3,2025-09-02,Luxury Apartment Stunning View Casablanca Invoice,$359.40
7,HMDKPHMKEH,Huésped anterior,Daniel Velasquez,NaN,2,0,0,27/8/2025,28/8/2025,1,2025-08-26,Luxury Suite | Casa Blanca Alta | Pool | Invoice,$92.65
10,HM4PCKWSE9,Huésped anterior,José Jaramillo Aguilar,NaN,4,1,0,23/8/2025,24/8/2025,1,2025-08-10,Luxury Apartment Stunning View Casablanca Invoice,$125.65
12,HMD4FQ3CDF,Huésped anterior,Gabriel Osorio,NaN,2,2,0,21/8/2025,23/8/2025,2,2025-08-16,Luxury Apartment Stunning View Casablanca Invoice,$230.36
13,HMEFFK5TE8,Huésped anterior,Diana Toledo,NaN,4,1,0,19/8/2025,21/8/2025,2,2025-08-12,Luxury Apartment Stunning View Casablanca Invoice,$209.42
15,HMBEEDNE5M,Huésped anterior,Tammy Bolaños,NaN,3,0,0,17/8/2025,19/8/2025,2,2025-08-15,Luxury Apartment Stunning View Casablanca Invoice,$209.42
16,HMCCRCFHH3,Huésped anterior,Veronica Jeaneth Ona Serrano,NaN,3,0,0,16/8/2025,17/8/2025,1,2025-08-13,Luxury Suite | Casa Blanca Alta | Pool | Invoice,$98.00
17,HM4J52A9KF,Huésped anterior,Frank Gonzalez,NaN,4,0,0,15/8/2025,16/8/2025,1,2025-08-15,Aurora | Vista Increíble | Depto Lujo | Factura,$97.92
